## PNL
Salazar Vega Rodrigo


contexto_words = ('dar','estado') si la palabra se encuentra 8 veces, vamos a meter 24, la frecuencia va a reflejar la frecuencia de todas las palabras
contexto_vector = (3, 2, 10, 6)


Para quitar las palabras repetidas hacemos normalización de todo.

Usamos numpy para hacer operaciones con vectores, se suman todos los números del vector de contexto $(sum(np.array(\begin{bmatrix}3, & 2,&10,&6,\end{bmatrix})))$ y dividimos el resultado por cada elemento del vector $vector\_ prob = (\frac{3}{21}, \frac{2}{21}, \frac{10}{21}, \frac{6}{21})$ la suma va a dar, es una distribución de todo el texto y luego se realiza el calculo para ver si son similares o no.

|   |context w1|context w2 | .... | ....|
|---|----------|-----------|------|-----|
|w1|            |           |       |   |
|w2|            |           |       |   |
|--|------------|-----------|-------|---|
|wn|0|  |   |   |

Se puede usar fredisq para representar. El vector de numero es la columna *context  w1*

El coseno del angulo es el vector normalizado $cos_\alpha = \frac{\overrightarrow{x}}{\overrightarrow{|x|}} * \frac{\overrightarrow{y}}{\overrightarrow{|y|}}$

$len(\frac{\overrightarrow{x}}{\overrightarrow{|x|}} = 1)$

Versión dos hay que tokenizar por parrafos

In [61]:
def similar_words(text,word,window=8, probability=True, tf_idf=False,dot_product=True,cosine=True):
    import numpy as np
    """
    parameters
    ----------
    text: list of words
    word: a word to wich similarity is measure of all other words in text
    """

    vocabulary = sorted(list(set(text)))
    contexts={}
    for w in vocabulary:
        context = []
        for i in range(len(text)):
            if text[i] == w:
                for j in range(i -int(window/2), i):
                    if j >= 0:
                        context.append(text[j])
                try:
                    for j in range(i +1, i+(int(window/2))+1):
                        context.append(text[j])
                except IndexError:
                    pass
        contexts[w] = context

    

    if probability:
        vectors = {}
        # counter = 0
        for v in vocabulary:
            try:
                context= contexts[v]
            except KeyError:
                pass
            vector = []
            for voc in vocabulary:
                vector.append(context.count(voc))
            vector = np.array(vector)
            s = np.sum(vector)
            vector = vector/s
            vectors[v] = vector
            

    if dot_product:
        similarities = {}
        try:
            v = vectors[word]
            for w in vectors.keys():
                similarities[w] = np.dot(vectors[w], v)
            similarities = (sorted(similarities.items(), key=lambda item: item[1], reverse=True))
            with open('similar_words_with_dot_product_of_prob_vectors_to_' + word + '.txt', 'w', encoding='UTF-8') as f:
                for item in similarities:
                    f.write(str(item)+'\n')
        except Exception as e:
            print('Ocurrió el siguiente error: ', e)
        return list(similarities)


    if cosine:
        similarities = {}
        v1 = vectors[word]
        for w in vectors.keys():
            v2 = vectors[w]
            similarities[w] = np.dot(v1, v2) /(np.linalg.norm(v1)*np.linalg.norm(v2))
        similarities = (sorted(similarities.items(), key= lambda item:item[1], reverse=True))
        with open('similar_words_with_cosine_of_prob_vectors_to_'+ word + '.txt', 'w', encoding= 'utf-8') as f:
            for item in similarities:
                f.write(str(item) + '\n')
        return list(similarities)

def s_dot_product(word, vectors, aux_path = ''):
    """
        Here you can get the similaritie of each word.
        word: the word you want to get similaritie.
        vectors: the vector space of your vocabulary.
        aux_path: the name you want to add to the similarities.
    """
    similarities = dict()
    v = vectors[word]
    for w in vectors.keys():
        similarities[w] = np.dot(vectors[w], v)
    similarities = (sorted(similarities.items(), key = lambda item: item[1], reverse = True))
    with open('./dot_product/similar_to_' + word + aux_path + '.txt', 'w', encoding = 'utf-8') as f:
        for item in similarities:
            f.write(str(item) + '\n')
    return list(similarities)


def s_cosine(word, vectors, aux_path = ''):
    import numpy as np
    """
        Here you can get the similaritie of each word.
        word: the word you want to get similaritie.
        vectors: the vector space of your vocabulary.
        aux_path: the name you want to add to the similarities.
    """
    similarities = dict()
    v = vectors[word]
    for w in vectors.keys():
        v2 = vectors[w]
        if np.linalg.norm(v) == 0 or np.linalg.norm(v2) == 0:
            similarities[w] = 0
        else:
            similarities[w] = np.dot(v, v2) / (np.linalg.norm(v) * np.linalg.norm(v2))
    similarities = (sorted(similarities.items(), key = lambda item: item[1], reverse = True))
    with open('./cosine/similar_to_' + word + aux_path + '.txt', 'w', encoding = 'utf-8') as f:
        for item in similarities:
            f.write(str(item) + '\n')
    return list(similarities)


def similar_words2(word, vectors, aux_path = '', tf_idf = False, dot_product = False, cosine = False):   
    """
        The Pipline to get similarities.
        word: the word you want to get similaritie.
        vectors: the vector space of your vocabulary.
        aux_path: the name you want to add to the similarities.
        dot_product: boolean, if you want to get similarities by this method.
        cosines: boolean, if you want to get similarities by this method.
        tf_idf: boolean, if you want to get similarities by this method.
    """     
    if dot_product:
        s_dot_product(word, vectors, aux_path)
                
    if cosine:
        s_cosine(word, vectors, aux_path)

In [17]:
def normalize(path = './../EXCELSIOR_100_files/'):
    from nltk.corpus import PlaintextCorpusReader
    from bs4 import BeautifulSoup
    import re
    from pickle import load
    # Obtenemos el corpus del directorio...
    corpus = PlaintextCorpusReader(path, '.*')
    file_list = corpus.fileids()
    # Juntamos todo el texto de todos los archivos...
    all_text = ''
    for file in file_list:
        with open(path + file, encoding = 'utf-8') as rfile:
            text = rfile.read()
            all_text += text
    # Removemos las etiquetas html...
    soup = BeautifulSoup(all_text, 'lxml')
    clean_text = soup.get_text()
    clean_text = clean_text.lower()
    # Tokenizamos el texto...
    words = clean_text.split()
    alphabetic_words = []
    for word in words:
        token = []
        for character in word:
            if re.match(r'^[a-záéíóúñü+$]', character):
                token.append(character)
        token = ''.join(token)
        if token != '':
            alphabetic_words.append(token)
    # Quitamos las Stop words...
    with open('./stopwords_es.txt', encoding = 'utf-8') as f:
        stop_words = f.readlines()
        stop_words = [w.strip() for w in stop_words]
    final_words = [word for word in alphabetic_words if word not in stop_words]
    print('Fin de la normalización...')
    return final_words

def lematize(text):
    from nltk.corpus import PlaintextCorpusReader
    from bs4 import BeautifulSoup
    import re
    from pickle import load
    lemmas = dict()
    with open('./generate.txt', encoding = 'latin-1') as file:
        lines = file.readlines()
        lines = [w.strip() for w in lines]
        for line in lines:
            line = line.strip()
            if line != '':
                words = line.split()
                token = words[0].strip()
                token = token.replace('#', '')
                lemma = words[-1].strip()
                lemmas[token] = lemma
    lemmatized_text = []
    for word in text:
        if word in lemmas.keys():
            lemmatized_text.append(lemmas[word])
        else:
            lemmatized_text.append(word)
    return lemmatized_text

In [19]:
try:
    text = normalize()
    print(f'Algunas palabras despues de la normalización: \n{text[:200]}')
except Exception as e:
    print('Ocurrió el siguiente error: ', e)

Fin de la normalización...
Algunas palabras despues de la normalización: 
['emodhtm', 'httpwwwexcelsiorcommxarthtml', 'excelsior', 'editorial', 'martes', 'abril', 'monstruosa', 'diferencia', 'colosistas', 'colosismo', 'luis', 'gutierrez', 'gonzalez', 'luis', 'gutiérrez', 'sotomayor', 'federico', 'arreola', 'colosistas', 'cabales', 'según', 'dijo', 'amigo', 'luis', 'donaldo', 'ciertamente', 'nombre', 'circunstancias', 'luis', 'donaldo', 'colosio', 'llenado', 'insistentemente', 'volúmenes', 'espacios', 'medios', 'comunicación', 'renovada', 'actualidad', 'padecido', 'frenético', 'vaivén', 'ficciones', 'judiciales', 'políticas', 'integran', 'disgregan', 'metafísicas', 'metafísicas', 'aún', 'luis', 'donaldo', 'desprende', 'envuelve', 'lado', 'espejo', 'dos', 'años', 'eternos', 'insolvencias', 'dale', 'dale', 'fantasía', 'magia', 'dónde', 'quedó', 'bolita', 'traído', 'pueblo', 'hastío', 'cansancio', 'inminencia', 'percibe', 'váyanse', 'diablo', 'quórum', 'nacional', 'veía', 'decidido', 'inst

In [31]:
try:
    texts = lematize(text)
    print(f'Algunas palabras despues de la lematización: \n{texts[:200]}')
except Exception as e:
    print('Error:', e)

Algunas palabras despues de la lematización: 
['emodhtm', 'httpwwwexcelsiorcommxarthtml', 'excelsior', 'editorial', 'martes', 'abril', 'monstruoso', 'diferencia', 'colosistas', 'colosismo', 'luis', 'gutierrez', 'gonzalez', 'luis', 'gutiérrez', 'sotomayor', 'federico', 'arreola', 'colosistas', 'cabal', 'según', 'decir', 'amigo', 'luis', 'donaldo', 'ciertamente', 'nombre', 'circunstancia', 'luis', 'donaldo', 'colosio', 'llenar', 'insistentemente', 'volumen', 'espacio', 'medio', 'comunicación', 'renovar', 'actualidad', 'padecer', 'frenético', 'vaivén', 'ficción', 'judicial', 'política', 'integrar', 'disgregar', 'metafísicas', 'metafísicas', 'aún', 'luis', 'donaldo', 'desprender', 'envolver', 'lado', 'espejo', 'dos', 'año', 'eterno', 'insolvencias', 'dar', 'dar', 'fantasía', 'magia', 'dónde', 'quedar', 'bolita', 'traer', 'pueblo', 'hastío', 'cansancio', 'inminencia', 'percibir', 'ir', 'diablo', 'quórum', 'nacional', 'ver', 'decidido', 'instalar', 'sécula', 'seculórum', 'demanda', 'justicia

In [62]:
m = 10
window = 8
word = 'liberación'

dot_prod = similar_words(texts,word,window=8, probability=True, tf_idf=True,dot_product=True,cosine=False)
cosine = similar_words(texts,word,window=8, probability=True, tf_idf=True,dot_product=False,cosine=True) 


In [68]:
import nltk

In [69]:

dot_prod_eti = dict(dot_prod).keys()
nltk.pos_tag(dot_prod_eti)


[('liberación', 'NN'),
 ('banxico', 'NN'),
 ('desfase', 'NN'),
 ('horno', 'NN'),
 ('microondas', 'NNS'),
 ('monitor', 'VBP'),
 ('belén', 'JJ'),
 ('bimestre', 'NN'),
 ('cinescopios', 'NNS'),
 ('lentamente', 'VBP'),
 ('ligeramente', 'NN'),
 ('televisor', 'NN'),
 ('ésto', 'NNP'),
 ('captación', 'NN'),
 ('inyectar', 'NN'),
 ('sintonizadores', 'VBZ'),
 ('ascender', 'JJR'),
 ('agricultor', 'NN'),
 ('ahuyentar', 'NN'),
 ('apaciguar', 'NN'),
 ('arca', 'NN'),
 ('austria', 'NN'),
 ('cautivo', 'NN'),
 ('constantino', 'NN'),
 ('contraloría', 'NN'),
 ('cárnico', 'NN'),
 ('denominar', 'NN'),
 ('derramar', 'NN'),
 ('desviar', 'NN'),
 ('dispersar', 'NN'),
 ('dispositivo', 'NN'),
 ('domínguez', 'NN'),
 ('equivalente', 'NN'),
 ('especulativos', 'FW'),
 ('fresca', 'NN'),
 ('ilegalmente', 'NN'),
 ('inestabilidad', 'NN'),
 ('inverlat', 'JJ'),
 ('levantamiento', 'NN'),
 ('litro', 'NN'),
 ('n$', 'JJ'),
 ('pantelhó', 'JJ'),
 ('patrimonial', 'JJ'),
 ('procampo', 'NN'),
 ('quinta', 'NN'),
 ('rubro', 'NN'),
 ('s